In [1]:
# read txt document of entry points
# also iterate over all tif files (adult 3D mask images) in the same path and perform analysis 
# all analysis except column analysis 

For explanation of the the individual columns in the result csv file, expand the next cell

In [2]:
"""" Results' columns' description 
'neuron':  neuron name extracted from file name
'subtype': neuron name extracted from file name
'entry_point': entry point of the neuron, read from seperate txt file

'volume': number of pixels in the 3D masked image
'net_area': number of pixels in the 2D masked image (the 2D is a sum projection of the 3D image)
'area': area of the convex_hull surronding the 2D image (obtained from skimage.measure.regionprops)
'AP_spread': (max-min) of value pixels index in last (x) dimension of the 2D image
'DV_spread': (max-min) of value pixels index in (y) dimension of the 2D image

'density_2D': solidity value obtained from skimage.measure.regionprops on the 2D image
'density_3D': pixel count of 3D image/pixel count of 3D convexhull 
'object_perimeter': object perimenter value obtained from skimage.measure.regionprops on the 2D image
'convex_perimeter': perimeter of the surronding convex_hull

'circularity': (4*np.pi*convex_area)/(convex_perimeter**2)
'compactness': object_perimeter**2/(4*np.pi*net_area)
'surface_area': mesh_surface_area of the marching_cubes on the 3D masked image
'eccentricity': eccentricity value obtained from skimage.measure.regionprops on the 2D image 
>>>> change to 3D >>>>  'convexity': convex_perimeter/object_perimeter (maybe try surface_area/3D_image_convex_surface area)
'orientation': orientation value obtained from skimage.measure.regionprops on the 2D image 
'feret_diameter_max': feret_diameter_max value obtained from skimage.measure.regionprops on the 2D image 
'DGI':

'centroidZ': Z position of the centroid from regionprops on the 3D masked image 
'centroidY': Y position of the centroid from regionprops on the 3D masked image 
'centroidX': X position of the centroid from regionprops on the 3D masked image 
'cen_EP_dis': ecludian distance of the centroid from the entry point
'cen_EP_angle': angle of the line between centroid from the entry point (np.arctan2)

£££ PCA analysis
'asymmetry':
'xP_weight', 'xP_scale','xP_angle','xP_xy':
'xN_weight','xN_scale','xN_angle','xN_xy',
'yP_weight','yP_scale','yP_angle','yP_xy',
'yN_weight','yN_scale','yN_angle','yN_xy',
'PCA_angle':
'PCA_xy':
'PCA_shift_centroid':
"""

'" Results\' columns\' description \n\'neuron\':  neuron name extracted from file name\n\'subtype\': neuron name extracted from file name\n\'entry_point\': entry point of the neuron, read from seperate txt file\n\n\'volume\': number of pixels in the 3D masked image\n\'net_area\': number of pixels in the 2D masked image (the 2D is a sum projection of the 3D image)\n\'area\': area of the convex_hull surronding the 2D image (obtained from skimage.measure.regionprops)\n\'AP_spread\': (max-min) of value pixels index in last (x) dimension of the 2D image\n\'DV_spread\': (max-min) of value pixels index in (y) dimension of the 2D image\n\n\'density_2D\': solidity value obtained from skimage.measure.regionprops on the 2D image\n\'density_3D\': pixel count of 3D image/pixel count of 3D convexhull \n\'object_perimeter\': object perimenter value obtained from skimage.measure.regionprops on the 2D image\n\'convex_perimeter\': perimeter of the surronding convex_hull\n\n\'circularity\': (4*np.pi*conv

In [3]:
from tqdm import tqdm
import os
import re
import tifffile as tif

import utils.analysis as analysis
import utils.datautils as datautils

import numpy as np
from skimage.measure import regionprops, marching_cubes, mesh_surface_area, perimeter
import pandas as pd
import math
from scipy.spatial import ConvexHull

import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patches as mpatches
# import seaborn as sns
%matplotlib inline

from timeit import default_timer as timer
from multiprocessing import cpu_count, Process, Pool

In [36]:
#  input and save paths
data_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/'
save_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/output/'
subtype = 'D'
# stab_limit = 5 #(No. of timepoints for stable branches calculation)
start_age = 37

In [37]:
# getting list of folders in data_path, where each folder has files for one neuron
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if '_P' in f]
N_folders = sorted(N_folders)
N_folders

['211107_P36_N3', '220127_P36_N1']

In [38]:
# reading erntry_points file and converting it to dictionary of neu:[z,y,x]
EP_file = '/home/tli_project/Desktop/Data/T4/neurons/TLI_stage1_entry_points'
with open(EP_file) as f:
    entry_points = f.readlines()
entry_points = [l for l in entry_points if ': ' in l]
entry_points = [l.rstrip('\n').split(':') for l in entry_points]
entry_points = {l[0]: l[1].split(',') for l in entry_points}
entry_points = {neu:[int(x) for x in val] for neu,val in entry_points.items()}

In [39]:
results_cols = ['timepoint', 'volume', 'net_area', 'area',
                'AP_spread', 'DV_spread',
                'density_2D', 'density_3D', 'object_perimeter', 'convex_perimeter', 
                'circularity', 'compactness', 'surface_area',
                'eccentricity', 'convexity_2D', 'convexity_3D', 'orientation',
                'feret_diameter_max',
                'DGI',
                'centroidZ','centroidY', 'centroidX', 
                'cen_EP_dis', 'cen_EP_angle', 'cen_EP_Y','cen_EP_X',
                'asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry',
                'xP_weight','xP_scale','xP_angle','xP_xy', #all of this part is for the results from PCA_analysis
                'xN_weight','xN_scale','xN_angle','xN_xy',
                'yP_weight','yP_scale','yP_angle','yP_xy',
                'yN_weight','yN_scale','yN_angle','yN_xy',
                'PC1_angle','PC1_x','PC1_y']
# results_df = pd.DataFrame(columns=results_cols)
# for i in range(100):
#     results_df.loc[start_age+i*0.25] = [None for col in results_df.columns]

In [40]:
for i, n_file in tqdm(enumerate(N_folders)):
    neuron_results = pd.DataFrame(columns=results_cols)
    entry_point = entry_points[n_file][1:]
    start_im = re.search('P(.*)_N', n_file)
    start_im = float(start_im.group(1))
    start_ana = int((start_age - start_im)/0.25)
    files_list = datautils.get_file_names(data_path+n_file, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='')
    neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
    neuron = tif.imread(neu_path)
    mask_path = [f for f in files_list if 'mask' in f.lower()][0]
    mask = tif.imread(mask_path)
    neuron[mask==0] = 0
    neuron = neuron[start_ana:] #remove timepoints before start_ana from the 4D image array
    neuron[neuron != 0] = 1
    neu_2D = neuron.max(axis=1)
    neu_2D = neuron.max(axis=1)
    neuron_results['timepoint'] = [start_age+i*0.25 for i in range(len(neu_2D))]
    neuron_results['volume'] = neuron.sum(axis=tuple(np.arange(len(neuron.shape))[1:]))
    neuron_results['net_area'] = neu_2D.sum(axis=tuple(np.arange(len(neu_2D.shape))[1:]))

    reg_prop_2D = []
    reg_prop_3D = []
    for t, stack in enumerate(neuron):
        reg_prop_2D.append(regionprops(neu_2D[t])[0])
        reg_prop_3D.append(regionprops(stack)[0])
        img_PC = reg_prop_3D[t].coords
        neuron_results.loc[t,'DV_spread'] = (img_PC[:,1].max() - img_PC[:,1].min())*0.076
        neuron_results.loc[t,'AP_spread'] = (img_PC[:,2].max() - img_PC[:,2].min())*0.076

        verts, faces, normals, values = marching_cubes(stack, step_size=2, spacing=(0.4,0.076,0.076))
        neuron_results.loc[t,'surface_area'] = mesh_surface_area(verts, faces)
        convex_img_3D = reg_prop_3D[t].image_convex.astype('uint8')
        verts, faces, normals, values = marching_cubes(convex_img_3D, step_size=2, spacing=(0.4,0.076,0.076))
        neuron_results.loc[t,'convexity_3D'] = mesh_surface_area(verts, faces)/neuron_results.loc[t,'surface_area']

        asymmetry_values, PCA_df, shifted_coor = analysis.metric_dump(neu_2D[t],entry_point[1:],plot=False)
        asymmetries = ['asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry']
        neuron_results.loc[t,asymmetries]  = asymmetry_values

        PCA_res = ['xP_weight','xP_scale','xP_angle','xP_xy',
                    'xN_weight','xN_scale','xN_angle','xN_xy',
                    'yP_weight','yP_scale','yP_angle','yP_xy',
                    'yN_weight','yN_scale','yN_angle','yN_xy']
        PCA_arr = np.array(PCA_df.loc[:, PCA_df.columns != 'axis'])
        neuron_results.loc[t,PCA_res]  = PCA_arr.ravel()
        neuron_results.loc[t,'PC1_angle'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].Angle
        neuron_results.loc[t,'PC1_x'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[0]
        neuron_results.loc[t,'PC1_y'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[1]

        neuron_results.loc[t,['centroidZ','centroidY','centroidX']] = reg_prop_3D[t].centroid
        centroid_EP = np.array(entry_point)-np.array(reg_prop_3D[t].centroid)
        neuron_results.loc[t,'cen_EP_dis'] = np.linalg.norm(centroid_EP[1:])
        neuron_results.loc[t,['cen_EP_Y','cen_EP_X']] = centroid_EP[1:]
        p1 = (centroid_EP[1],centroid_EP[2])
        neuron_results.loc[t,'cen_EP_angle'] = np.rad2deg((np.arctan2(*p1[::-1])) % (2 * np.pi))


    del neuron, neu_2D
    neuron_results['density_2D'] = [r.solidity for r in reg_prop_2D]
    neuron_results['density_3D'] = [r.area_convex for r in reg_prop_3D]
    neuron_results['density_3D'] = neuron_results.loc[t,'volume']/neuron_results['density_3D']
    neuron_results['area'] = [r.area_convex for r in reg_prop_2D]
    neuron_results['orientation'] = [r.orientation for r in reg_prop_2D]
    neuron_results['eccentricity'] = [r.eccentricity for r in reg_prop_2D]
    neuron_results['feret_diameter_max'] = [r.feret_diameter_max for r in reg_prop_2D]
    neuron_results['convex_perimeter'] = [perimeter(r.image_convex) for r in reg_prop_2D]
    neuron_results['object_perimeter'] = [r.perimeter for r in reg_prop_2D]
    neuron_results['convexity_2D'] = neuron_results['convex_perimeter']/neuron_results['object_perimeter']
    neuron_results['circularity'] = (4*np.pi*neuron_results['area'])/(neuron_results['convex_perimeter']**2)
    neuron_results['compactness'] = neuron_results['object_perimeter']**2/(4*np.pi*neuron_results['net_area'])
    del reg_prop_2D, reg_prop_3D

    csv_name = save_path+n_file+'_s'+subtype+'_new_1.csv'
    neuron_results.to_csv(csv_name)

0it [00:00, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/211107_P36_N3/211107_P36_N3_R250_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/211107_P36_N3/211107_P36_N3_R250_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/211107_P36_N3/211107_P36_N3_R250_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/211107_P36_N3/211107_P36_N3_R250_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/211107_P36_N3/211107_P36_N3_CropRoi.roi']


1it [01:28, 88.86s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/220127_P36_N1/220127_P36_N1_Z30_R240_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/220127_P36_N1/220127_P36_N1_Z30_R240_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/220127_P36_N1/220127_P36_N1_Z30_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/220127_P36_N1/220127_P36_N1_R240_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeD/220127_P36_N1/220127_P36_N1_R240_JE_mask.tif']


2it [02:11, 65.52s/it]


In [41]:
len(neuron_results)

89

In [8]:
def process_image(n_file):
    neuron_results = pd.DataFrame(columns=results_cols)
    entry_point = entry_points[n_file][1:] # remove the first T element
    start_im = re.search('P(.*)_N', n_file)
    start_im = float(start_im.group(1))
    start_ana = int((start_age - start_im)/0.25)
    files_list = datautils.get_file_names(data_path+n_file, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='')
    neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
    neuron = tif.imread(neu_path)
    mask_path = [f for f in files_list if 'mask' in f.lower()][0]
    mask = tif.imread(mask_path)
    neuron[mask==0] = 0
    neuron = neuron[start_ana:] #remove timepoints before start_ana from the 4D image array
    neuron[neuron != 0] = 1
    neu_2D = neuron.max(axis=1)
    neuron_results['timepoint'] = [start_age+i*0.25 for i in range(len(neu_2D))]
    neuron_results['volume'] = neuron.sum(axis=tuple(np.arange(len(neuron.shape))[1:]))
    neuron_results['net_area'] = neu_2D.sum(axis=tuple(np.arange(len(neu_2D.shape))[1:]))
    for t, stack in enumerate(neuron):
        reg_prop_2D = regionprops(neu_2D[t])[0]
        reg_prop_3D = regionprops(stack)[0]
        neuron_results.loc[t,'density_2D'] = reg_prop_2D.solidity
        neuron_results.loc[t,'area'] = reg_prop_2D.area_convex
        neuron_results.loc[t,'orientation'] = reg_prop_2D.orientation
        neuron_results.loc[t,'eccentricity'] = reg_prop_2D.eccentricity
        neuron_results.loc[t,'feret_diameter_max'] = reg_prop_2D.feret_diameter_max
        neuron_results.loc[t,'convex_perimeter'] = perimeter(reg_prop_2D.image_convex)
        neuron_results.loc[t,'object_perimeter'] = reg_prop_2D.perimeter
        neuron_results.loc[t,['centroidZ','centroidY','centroidX']] = reg_prop_3D.centroid
        neuron_results.loc[t,'density_3D'] = neuron_results.loc[t,'volume']/reg_prop_3D.area_convex
        img_PC = reg_prop_3D.coords
        neuron_results.loc[t,'DV_spread'] = (img_PC[:,1].max() - img_PC[:,1].min())*0.076
        neuron_results.loc[t,'AP_spread'] = (img_PC[:,2].max() - img_PC[:,2].min())*0.076
        centroid_EP = np.array(entry_point)-np.array(reg_prop_3D.centroid)
        neuron_results.loc[t,'cen_EP_dis'] = np.linalg.norm(centroid_EP[1:])
        neuron_results.loc[t,['cen_EP_Y','cen_EP_X']] = centroid_EP[1:]
        p1 = (centroid_EP[1],centroid_EP[2])
        neuron_results.loc[t,'cen_EP_angle'] = np.rad2deg((np.arctan2(*p1[::-1])) % (2 * np.pi))

        verts, faces, normals, values = marching_cubes(stack, step_size=2, spacing=(0.4,0.076,0.076))
        neuron_results.loc[t,'surface_area'] = mesh_surface_area(verts, faces)
        convex_img_3D = reg_prop_3D.image_convex.astype('uint8')
        verts, faces, normals, values = marching_cubes(convex_img_3D, step_size=2, spacing=(0.4,0.076,0.076))
        neuron_results.loc[t,'convexity_3D'] = mesh_surface_area(verts, faces)/neuron_results.loc[t,'surface_area']

        asymmetry_values, PCA_df, shifted_coor = analysis.metric_dump(neu_2D[t],entry_point[1:],plot=False)
        asymmetries = ['asymmetry', 'x_asymmetry', 'y_asymmetry', 'PC1_asymmetry', 'PC2_asymmetry']
        neuron_results.loc[t,asymmetries]  = asymmetry_values

        PCA_res = ['xP_weight','xP_scale','xP_angle','xP_xy',
                    'xN_weight','xN_scale','xN_angle','xN_xy',
                    'yP_weight','yP_scale','yP_angle','yP_xy',
                    'yN_weight','yN_scale','yN_angle','yN_xy']
        PCA_arr = np.array(PCA_df.loc[:, PCA_df.columns != 'axis'])
        neuron_results.loc[t,PCA_res]  = PCA_arr.ravel()
        neuron_results.loc[t,'PC1_angle'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].Angle
        neuron_results.loc[t,'PC1_x'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[0]
        neuron_results.loc[t,'PC1_y'] = PCA_df.loc[PCA_df.Fraction_weight.idxmax()].xy[1]
        # neuron_results.loc[t,'PC1_shift_centroid'] = (sum(shifted_coor[:,1]) / len(shifted_coor), sum(shifted_coor[:,0]) / len(shifted_coor))
        
        neuron_results.loc[t,'DGI'] = analysis.DGI_3D(stack,entry_point)

    neuron_results['convexity_2D'] = neuron_results['convex_perimeter']/neuron_results['object_perimeter']
    neuron_results['circularity'] = (4*np.pi*neuron_results['area'])/(neuron_results['convex_perimeter']**2)
    neuron_results['compactness'] = neuron_results['object_perimeter']#     p = Pool(cpu_count()-1)
    csv_name = save_path+n_file+'_s'+subtype+'.csv'
    neuron_results.to_csv(csv_name)
    return neuron_results

# if __name__ == '__main__':
#     # c = cpu_count()-1
#     with Pool(3) as p:
#         p.map(process_image, N_folders)

for n_file in tqdm(N_folders):
    process_image(n_file)

  0%|          | 0/9 [00:00<?, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_ColROIs.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_5D_new.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_5D.tif']


 11%|█         | 1/9 [02:44<21:59, 164.91s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211113_P36_N2/211113_P36_N2_new_CropRoi.roi', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211113_P36_N2/211113_P36_N2_FlipV_R245_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211113_P36_N2/211113_P36_N2_FlipV_R245_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211113_P36_N2/211113_P36_N2_FlipV_R245_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211113_P36_N2/211113_P36_N2_FlipV_R245_5D_new.tif']


 22%|██▏       | 2/9 [04:42<15:57, 136.84s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_clahe1.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_CropRoi.roi']


 33%|███▎      | 3/9 [07:44<15:44, 157.50s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R277_JE_seg.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R272_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R272_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R272_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_CropRoi.roi']


 33%|███▎      | 3/9 [07:50<15:41, 156.95s/it]


IndexError: list index out of range

In [9]:
neuron_results.head()

NameError: name 'neuron_results' is not defined